In [3]:
from ultralytics import YOLO
# from yaen.yaen import YAEN
import cv2
import json
import torch
# from yaen.models.baselines import *

# Load YOLOv9
device_name = 'cpu'
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    device_name = '0'
    yolov9 = YOLO("yolov9c.pt").to('cuda')
    print('Detecting using GPU...')
else:
    device_name = 'cpu'
    yolov9 = YOLO("yolov9c.pt").to('cpu')
    print('Detecting using CPU...')

Detecting using GPU...


In [2]:
from deepbox.deepbox import Deepbox

deepbox = Deepbox()




Instructions for updating:
dim is deprecated, use axis instead


In [26]:
im = cv2.imread(r'images\input3.png')
result = yolov9.predict(im)[0]
detc_2ds = []
for i, box in enumerate(result.boxes):
    accepted_class_names = ['car', 'bus', 'truck']
    class_index = int(result.boxes.cls[i])
    if result.names[class_index] not in accepted_class_names:
        continue

    conf = float(result.boxes.conf[i])
    xywh = result.boxes.xywh[i]
    x1 = int(xywh[0] - xywh[2]/2)
    y1 = int(xywh[1] - xywh[3]/2)
    x2 = int(xywh[0] + xywh[2]/2)
    y2 = int(xywh[1] + xywh[3]/2)
    detc_2ds.append([result.names[class_index], [x1, y1, x2, y2]])

yaws = deepbox.predict(im, detc_2ds)


0: 384x640 6 cars, 1 truck, 205.7ms
Speed: 288.6ms preprocess, 205.7ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
1/1 [==============================] - 0s 172ms/step


In [27]:
import numpy as np
yaw_degrees = [np.rad2deg(yaw) for yaw in yaws]

for i, detc_2d in enumerate(detc_2ds):
    cv2.rectangle(im, (int(detc_2d[1][0]), int(detc_2d[1][1])), (int(detc_2d[1][2]), int(detc_2d[1][3])), (0, 255, 0), 3)
    cv2.putText(im, f'{int(yaw_degrees[i])}, {detc_2d[0]}', (int(detc_2d[1][0]), int(detc_2d[1][1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

cv2.imwrite('deepbox_trial_3.png', im)
# from matplotlib import pyplot as plt
# plt.imshow(im)
# plt.show()

True